In [ ]:
# Install required packages if not already available.
# Uncomment and run the following line if packages are missing.
# Note: aif360 installation can take some time and may require a C compiler on some platforms.
# !pip install aif360==0.6.0 numpy pandas scikit-learn matplotlib seaborn

print('If needed, run: pip install -r requirements.txt or uncomment the pip install line in this cell')

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# AIF360 imports (may fail if not installed)
try:
    from aif360.datasets import CompasDataset
    from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
except Exception as e:
    print('AIF360 import failed:', e)
    print('If AIF360 is not installed, run: pip install aif360==0.6.0')

In [ ]:
# Load COMPAS dataset (AIF360 provided wrapper)
try:
    dataset = CompasDataset()
    print('Loaded COMPAS dataset with {} instances'.format(dataset.features.shape[0]))
    print('Protected attributes:', dataset.protected_attribute_names)
    print('Label names:', dataset.metadata['label_names'])
except Exception as e:
    print('Failed to load CompasDataset from aif360:', e)
    dataset = None

In [ ]:
# Compute baseline group metrics (disparate impact on labels / base rates)
if dataset is not None:
    # Define privileged/unprivileged groups for race (AIF360 encoding: race=1 is typically Caucasian)
    privileged_groups = [{dataset.protected_attribute_names[0]: 1}]
    unprivileged_groups = [{dataset.protected_attribute_names[0]: 0}]

    bldm = BinaryLabelDatasetMetric(dataset, privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
    print('Base rate (mean label) for privileged:', bldm.mean_positive_rate(privileged=True))
    print('Base rate (mean label) for unprivileged:', bldm.mean_positive_rate(privileged=False))
    print('Disparate impact (labels):', bldm.disparate_impact())
else:
    print('Dataset not available; skipping baseline metrics')

In [ ]:
# Train a simple classifier and evaluate fairness on predictions
if dataset is not None:
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split

    X = dataset.features
    y = dataset.labels.ravel()

    # Split (stratify to keep label distribution)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    scaler = StandardScaler().fit(X_train)
    X_train_s = scaler.transform(X_train)
    X_test_s = scaler.transform(X_test)

    clf = LogisticRegression(max_iter=1000).fit(X_train_s, y_train)
    y_pred = clf.predict(X_test_s)

    # Build AIF360 BinaryLabelDataset objects for test and predictions
    test_dataset = dataset.subset(np.where(dataset.instance_names.isin(dataset.instance_names))[0])[0:0] if False else None
    # Simpler: use the original dataset split approach via indices to construct test/set from AIF360 dataset
    # We'll create a shallow copy of dataset for test rows using indices from train_test_split on features.
    # NOTE: AIF360 dataset API is flexible; here we use the numpy split approach to find test indices.
    # Find indices of test set by matching rows (less robust but works for demonstration).
    from sklearn.neighbors import NearestNeighbors
    nbrs = NearestNeighbors(n_neighbors=1).fit(dataset.features)
    _, idxs = nbrs.kneighbors(X_test, return_distance=True)
    test_indices = idxs.ravel()

    aif_test = dataset.subset(test_indices)
    aif_pred = aif_test.copy()
    aif_pred.labels = y_pred.reshape(-1, 1)

    # Metrics
    cm = ClassificationMetric(aif_test, aif_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
    fpr_diff = cm.false_positive_rate_difference()
    eqopp_diff = cm.equal_opportunity_difference()
    print('False positive rate difference (unprivileged - privileged):', fpr_diff)
    print('Equal opportunity difference (TPR difference):', eqopp_diff)

    # Disparate impact on predictions
    bldm_pred = BinaryLabelDatasetMetric(aif_pred, privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
    print('Disparate impact (predictions):', bldm_pred.disparate_impact())
else:
    print('Dataset not available; skipping model training and prediction metrics')

In [ ]:
# Visualize False Positive Rate by group (privileged vs unprivileged)
if dataset is not None:
    groups = ['unprivileged', 'privileged']
    fpr_unpriv = cm.false_positive_rate(privileged=False)
    fpr_priv = cm.false_positive_rate(privileged=True)
    fig, ax = plt.subplots(figsize=(6,4))
    sns.barplot(x=groups, y=[fpr_unpriv, fpr_priv], palette='viridis', ax=ax)
    ax.set_ylabel('False Positive Rate')
    ax.set_title('False Positive Rate by Group')
    plt.show()
else:
    print('Dataset not available; skipping visualization')

## Next steps and remediation ideas
- If bias is observed, try pre-processing (reweighing), in-processing (adversarial debiasing), or post-processing (calibrated equalized odds) algorithms from AIF360.
- Include domain experts and community stakeholders when choosing fairness definitions and thresholds.
- Log and document dataset provenance, preprocessing steps, and evaluation results for accountability.